In [7]:
#| default_exp cross

In [8]:
#|export
import jax
import jax.numpy as jnp
from typing import Tuple
from chewc.meiosis import create_gamete
from chewc.structs import GeneticMap

@jax.jit
def cross_pair(
    key: jax.Array,
    mother_geno: jax.Array, # Shape: (n_chr, 2, n_loci)
    father_geno: jax.Array, # Shape: (n_chr, 2, n_loci)
    mother_ibd: jax.Array,  # Shape: (n_chr, 2, n_loci)
    father_ibd: jax.Array,  # Shape: (n_chr, 2, n_loci)
    genetic_map: GeneticMap
) -> Tuple[jax.Array, jax.Array]:
    """
    Creates one diploid offspring from two parents.

    Args:
        key: The JAX PRNGKey.
        mother_geno, father_geno: Genotype arrays for the parents.
        mother_ibd, father_ibd: IBD arrays for the parents.
        genetic_map: The GeneticMap object.

    Returns:
        A tuple of (offspring_geno, offspring_ibd).
        Shape: (n_chr, 2, n_loci).
    """
    key_mother, key_father = jax.random.split(key)
    
    # Create one gamete from each parent
    mother_gamete_geno, mother_gamete_ibd = create_gamete(
        key_mother, mother_geno, mother_ibd, genetic_map
    )
    father_gamete_geno, father_gamete_ibd = create_gamete(
        key_father, father_geno, father_ibd, genetic_map
    )
    
    # Stack the two gametes to form the diploid offspring
    offspring_geno = jnp.stack([mother_gamete_geno, father_gamete_geno], axis=1)
    offspring_ibd = jnp.stack([mother_gamete_ibd, father_gamete_ibd], axis=1)

    return offspring_geno, offspring_ibd

In [9]:
#| hide
import nbdev; nbdev.nbdev_export()